In [239]:
import networkx as nx
import numpy as np
import cv2
import json
import networkx as nx
import re
import argparse
import glob
import os

from route_generator import iter_all_white_points, draw_waypoint, rdp_algorithm, recorded_find_path
from add_path_to_graph import add_path, read_adjlist_with_tuples

##################### Input Town number #####################

town = int(input("Enter town number: 1, 2, 3, or 10"))

if town == 10:
    waypoint_map = cv2.imread("./waypoint_maps/Town10HD.png", 1)
    town_name = "Town10HD"
else:
    town_name = f"Town0{town}"
    waypoint_map = cv2.imread(f"./waypoint_maps/{town_name}.png", 1)


##################### Read latest graph list of the given town #####################

list_of_files = glob.glob(f'graph_list/{town_name}/*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)

if town == 10:
    index = int(latest_file[30:-8])
    new_file_name = latest_file[:30] + str(index+1) + latest_file[-8:]
else:
    index = int(latest_file[27:-8])
    new_file_name = latest_file[:27] + str(index+1) + latest_file[-8:]

G = read_adjlist_with_tuples(latest_file)

print(latest_file)

graph_list/Town05\5_graph_v299.adjlist


In [240]:
################# Change start, end point #################

reverse = input("Do reverse search or not? (y for reverse search, other for original search)\n")

start, end = (4647, 3294),(4652, 4264)

if reverse == "y":
    start, end = end, start


In [241]:
if start == (0,0) or end == (0,0):
    print("Didn't change to your point!!!!!!!!")

all_whites_pos = iter_all_white_points(waypoint_map)

new_img = waypoint_map.copy()
new_img[start[0]][start[1]] = np.array([0, 0, 255])
new_img[end[0]][end[1]] = np.array([0, 0, 255])

cv2.circle(new_img, start[::-1], radius=5, color=(255, 0, 0), thickness=5)
cv2.circle(new_img, end[::-1], radius=5, color=(0, 255, 0), thickness=5)


print("Start point:", start)
print("End point:", end)
print("Finding Path...")
# final_path = find_path(start, end, all_whites_pos)
flag, final_path = recorded_find_path(start, end, all_whites_pos, "Forward", False)

##### Path not found #####
if flag == False:
    print("Not found!!!")
    print(final_path)
    for i, j in final_path[0]:
        new_img[i][j] = np.array([255, 255, 0])
    
    draw_img = draw_waypoint(waypoint_map, start, end, final_path[0])
    rdp_img, _ = rdp_algorithm(draw_img, final_path[0])
    cv2.imwrite(f"test/not_found/test__{start}_{end}.png", new_img)
else:
    print("found!!!")
    print(final_path[0])
    print("Adding to the graph list...")
    
    if reverse == "y":
        fp = final_path[0][::-1]
        fp = [fp]
        add_path(G, fp, new_file_name)
    else:
        add_path(G, final_path, new_file_name)
    
    print(new_file_name, "Store!")
    draw_img = draw_waypoint(waypoint_map, start, end, final_path[0])
    rdp_img, _ = rdp_algorithm(draw_img, final_path[0])
    cv2.imwrite(f"test/found/test__{start}_{end}.png", rdp_img)

Start point: (4647, 3294)
End point: (4652, 4264)
Finding Path...
Get!!!
found!!!
[(4647, 3294), (4647, 3298), (4647, 3302), (4647, 3306), (4647, 3310), (4647, 3314), (4647, 3318), (4647, 3322), (4647, 3326), (4647, 3330), (4647, 3334), (4647, 3338), (4647, 3342), (4647, 3346), (4647, 3350), (4647, 3354), (4647, 3358), (4647, 3362), (4647, 3366), (4647, 3370), (4647, 3374), (4647, 3378), (4647, 3382), (4647, 3386), (4647, 3390), (4647, 3394), (4647, 3398), (4647, 3402), (4647, 3406), (4647, 3410), (4647, 3414), (4647, 3418), (4647, 3422), (4647, 3426), (4647, 3430), (4647, 3434), (4647, 3438), (4647, 3442), (4647, 3443), (4647, 3446), (4647, 3447), (4647, 3450), (4647, 3452), (4647, 3454), (4647, 3458), (4648, 3462), (4648, 3466), (4648, 3470), (4648, 3474), (4648, 3478), (4648, 3482), (4648, 3486), (4648, 3490), (4648, 3494), (4648, 3498), (4648, 3502), (4648, 3506), (4648, 3510), (4648, 3514), (4648, 3518), (4648, 3522), (4648, 3526), (4648, 3530), (4648, 3534), (4648, 3538), (4648, 

In [242]:
############### Quick test for other start point with the same goal point
start, end = (4647, 3294),(4652, 4264)

fp = nx.shortest_path(G,  start,  end)
draw_img = draw_waypoint(waypoint_map, start, end, fp)
rdp_img, _ = rdp_algorithm(draw_img, fp)
cv2.imwrite(f"test/check__{start}_{end}.png", rdp_img)
print(fp)

[(4647, 3294), (4647, 3302), (4647, 3314), (4647, 3318), (4647, 3330), (4647, 3334), (4647, 3346), (4647, 3350), (4647, 3362), (4647, 3366), (4647, 3378), (4647, 3382), (4647, 3394), (4647, 3398), (4647, 3410), (4647, 3414), (4647, 3426), (4647, 3430), (4647, 3442), (4647, 3446), (4647, 3452), (4645, 3461), (4648, 3466), (4648, 3478), (4648, 3482), (4648, 3494), (4648, 3498), (4648, 3510), (4648, 3514), (4648, 3526), (4648, 3530), (4648, 3542), (4648, 3546), (4648, 3558), (4648, 3562), (4648, 3574), (4648, 3578), (4648, 3590), (4648, 3594), (4648, 3606), (4648, 3610), (4648, 3622), (4648, 3626), (4648, 3638), (4648, 3642), (4648, 3654), (4648, 3658), (4649, 3670), (4649, 3674), (4649, 3686), (4649, 3690), (4649, 3702), (4649, 3706), (4649, 3715), (4649, 3718), (4649, 3722), (4649, 3728), (4649, 3740), (4649, 3744), (4649, 3756), (4649, 3760), (4649, 3772), (4649, 3776), (4649, 3788), (4649, 3792), (4649, 3804), (4649, 3808), (4649, 3820), (4649, 3824), (4649, 3836), (4649, 3840), (4649